<a href="https://colab.research.google.com/github/qq1767899088/VisualGo/blob/main/myTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ln -s /content/drive/MyDrive/visualGoDataset/VisualGo/ /content/myhome
%cd /content/myhome
!ls 

In [ ]:
!python myTensorFlow.py

In [ ]:
!cp ~/.keras/datasets/mnist.npz .

In [ ]:
!ls

In [ ]:
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)
b = numpy.load("mnist.npz")
print(b.files)

print(b["x_tes"t"][0])

In [ ]:
!python myParseData.py

In [ ]:

import json
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
from skimage.transform import resize
import cv2

from tqdm import tqdm, tqdm_notebook
from skimage.draw import polygon2mask, polygon, polygon_perimeter

from scripts.transformers import MaskTransformer, PerspectiveTransformer, ThreasholdTransformer

import torch
import torchvision.transforms as T


folder = "data/board_masks/upload/"

baseFile = "data/result.json"
image_size = 128
# define a dataset class for the Dataloaders
basePath = folder
imageSize = image_size

# load the dataset
with open(baseFile) as json_file:
    jsonData = json.load(json_file)

images = []

masks = []

# save every image and ground truth mask
p = jsonData[42]
data = p["data"]
imagePath = data["image"]
#img = np.array(Image.open(basePath + os.path.basename(imagePath)))
import matplotlib.image as mpimg
img = np.array(Image.open(basePath + os.path.basename(imagePath))) 
import matplotlib.pyplot as plt

y = np.array(p["completions"][0]["result"][0]["value"]["points"])
#mask = polygon2mask((800,800), y*8).astype(bool).T
print(img)
print(y)
plt.imshow(img)
plt.show()


In [ ]:
%pycat myParseData.py

In [ ]:
plt.imshow(img)

In [ ]:
%loadpy myParseData.py

In [ ]:
%lsmagic

In [ ]:
%load --help


In [ ]:
!python myCnnNet.py

In [ ]:
!pip install pytorch_model_summary


import torch
import torch.nn.functional as F
import torch.nn as nn

import matplotlib.pyplot as plt

import os

import numpy as np
from PIL import Image
from tqdm import tqdm

from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T

from scripts.unet import UNet
from scripts.transformers import MaskTransformer, PerspectiveTransformer, ThreasholdTransformer
from scripts.dataloaders import PositionDataset

from pytorch_model_summary import summary

import json

import tensorflow as tf
import tensorflow.keras as tf2
print(tf.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(in_channels=3, n_classes=2, wf=5, depth=4, padding=True, up_mode='upsample')
model.load_state_dict(torch.load("state_dicts/checkpoint.pth",device))
model.eval()
print(summary(model, torch.zeros((1, 3, 128, 128)), show_input=False))
pd = PositionDataset("data/goBoards/training/", 128, model)


In [ ]:
plt.imshow(pd[42][1])

In [ ]:
x3 = []
y3 = []
for i in pd:
	x3.append(i[0].tolist())
	y3.append(i[1].tolist())
import sys
np.set_printoptions(threshold=sys.maxsize)


In [ ]:
x1 = np.array(x3)
x = np.moveaxis(x1,1,3)
y1 = np.array(y3)
y = y1.reshape(y1.shape[0],-1)
print(x.shape)	
print(y1.shape)	

In [ ]:
plt.imshow(x[42][7:120,5:120])

In [ ]:
plt.imshow(y1[42])

In [ ]:

#myData = tf2.datasets.mnist
# y:ground truth;x:model argument ,ie tensors to be feed to the model function
#(x,y),(x2,y2) = myData.load_data()
#x,x2 = x/255.0,x2/255.0

#a = tf2.layers.Flatten(input_shape=(28,28) ) 
a = tf2.layers.Conv2D(128,kernel_size = 3,activation = "relu", input_shape=(128,128,3) ) 
a2 = tf2.layers.MaxPooling2D((2, 2)) 
a3 = tf2.layers.Conv2D(128,kernel_size = 3,activation = "relu") 
a4 = tf2.layers.Flatten() 


a5 = tf2.layers.Dense(128,activation='relu')
a6 = tf2.layers.Dropout(0.2) 
a7 = tf2.layers.Dense(169,activation='softmax') 
myModel = tf2.models.Sequential([a,a2,a3,a4,a5,a6,a7 ])
# p prediction
#p = myModel(x[:1]).numpy()
#p
#tf.nn.softmax(p).numpy()

lossFunction = tf2.losses.CategoricalCrossentropy(True)
#lossFunction(y[:1],p).numpy()
# "adam":optimizer;["accuracy"]:metrics
myModel.compile("adam",lossFunction,["accuracy"])
# 5:epochs
myModel.fit(x,y,5)


In [ ]:
!ls

In [ ]:
!pip install sgfmill

In [ ]:
import tensorflow.keras as tf2
import cv2 as cv
import numpy as np
import time

img1 = cv.imread('sgf1.png')/255.0
img2 = cv.imread('sgf2.png')/255.0
img3 = cv.imread('sgf3.png')/255.0
img4 = cv.imread('sgf4.png')/255.0
img5 = cv.imread('sgf5.png')/255.0
#cv.imshow('Original', img)
#time.sleep(20)

#img3 = img[0:1600,0:1600]
#img4 = img[227:1230,107:1020]
#img4 = img[335:1230,125:1020]

(x,y),(x2,y2)=(43,243),(1360,1560)
mycrop = img1[y:y2,x:x2]
l=(x2-x)/36
l2 = int(2*l)
def getP(x3,y3):
	a2 = int(x3-l)
	b2 = int(y3-l)
	return [a2,b2,a2+l2,b2+l2]
#mask = cv.circle(blank, center_coordinates, radius, 255, -1)
x_train = []
def myAppend(img5,x_train3):
	for b in range(19):
		for a in range(19):
	
			c = getP(x+2*l*a,y+2*l*b)
			d = img5[c[1]:c[3],c[0]:c[2]]
			x_train3.append(d)
myAppend(img1,x_train);
myAppend(img2,x_train);
myAppend(img3,x_train);
myAppend(img4,x_train);
myAppend(img5,x_train);

import pandas as pd
s = pd.Series(list('abca'))
s1=pd.get_dummies(s)
print(s1["a"])
from sgfmill import sgf
mydict = {"w":(1,0,0),"b":(0,0,1)}
def myEncode(z):
	return mydict[z]
import numpy as np
y_train = np.full((361*5,3),(0,1,0))
def parseSgf(u,v):
	with open(u, "rb") as f:
	    game = sgf.Sgf_game.from_bytes(f.read())
	winner = game.get_winner()
	board_size = game.get_size()
	root_node = game.get_root()
	b_player = root_node.get("PB")
	w_player = root_node.get("PW")
	for node in game.get_main_sequence():
	    colour, move=node.get_move()
	    #print(move)
	    if move is None:
	        continue
	    row, col = move
	    y_train[v*361+19*(18-row)+col]=myEncode(colour)

parseSgf("sgf1.sgf",0)
parseSgf("sgf2.sgf",1)
parseSgf("sgf3.sgf",2)
parseSgf("sgf4.sgf",3)
parseSgf("sgf5.sgf",4)

print(y_train)
#a = tf2.layers.Flatten(input_shape=(28,28) ) 
a = tf2.layers.Conv2D(128,kernel_size = 3,activation = "relu", input_shape=(l2,l2,3) ) 
a2 = tf2.layers.MaxPooling2D((2, 2)) 
a3 = tf2.layers.Conv2D(128,kernel_size = 3,activation = "relu") 
a4 = tf2.layers.Flatten() 
a2 = tf2.layers.Dense(128,activation = "relu" ) 
a3 = tf2.layers.Dropout(0.2) 
a5 = tf2.layers.Dense(3,activation = "softmax",) 
myModel = tf2.models.Sequential([a,a2,a3,a4,a5 ])
# p prediction

lossFunction = tf2.losses.CategoricalCrossentropy(False)
# "adam":optimizer;["accuracy"]:metrics
myModel.compile("adam",lossFunction,["accuracy"])
x_train2 = np.array(x_train)# 5:epochs
myModel.fit(x_train2,y_train,5)

w = 550
print(len(x_train)); 
#cv.imshow('Output', x_train[w])
print(y_train[w])
#cv.imwrite("321.png",resized)
#cv.imwrite("321.jpg",resized,[int(cv.IMWRITE_JPEG_QUALITY), 100])
#time.sleep(20)






In [ ]:
q=np.empty(1024)
q.reshape((-1,2,4,8)).shape

In [ ]:
mydict2 = [1,0,-1]
img9 = cv.imread('sgf9.png')/255.0
x_test = []
res = []
#def mydecode():
 #   decoded = tf.argmax(one_hot_encoded, axis=1)
myAppend(img9,x_test)

x_test3=np.array(x_test)
print(x_test3.shape)
x_test2=x_test3.reshape((-1,19,73,73,3))
for i in x_test2:
  #print(i)
  a5=myModel.predict(i)
  #print(a5)
  a6 = np.argmax(a5,axis=1)
  print(a6)
  a7 = map(lambda x:mydict2[x],a6)
  #print(list(a7))
  res.append(list(a7))
  print(res)



In [ ]:
#res2=np.reshape(np.array(res),(19,19))
res2 = res
print(res2)
plt.imshow(res2)

In [ ]:
plt.imshow(img9)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(x_train2[22][np.newaxis,...].shape)
t=361*4+19*9+9
plt.imshow(x_train2[t])
print(np.argmax(myModel.predict(x_train2[t][np.newaxis,...])[0]))

In [ ]:
!python myTensorFlow.py